# Advanced Functional Concepts

This notebook dives deeper into the `returns` library, exploring powerful patterns for handling more complex functional workflows. These tools help keep your code flat, readable, and robust, even when dealing with multiple operations that can fail.

## Pyromad of doom

In [98]:
"""
Pyramid of Doom Problem and Solution using the `returns` library

This example demonstrates:
1. The "Pyramid of Doom" problem with nested Result handling
2. Clean solution using the `returns` library's monadic operations
"""

import json
from typing import Any

from returns.pipeline import flow
from returns.pointfree import bind
from returns.result import Failure, Result, Success


# Simulate some operations that can fail
def parse_json(data: str) -> Result[dict[Any, Any], str]:
    """Parse JSON string, returning Result."""
    try:
        parsed = json.loads(data)
        return Success(parsed)
    except json.JSONDecodeError as e:
        return Failure(f"JSON parsing error: {str(e)}")


def extract_user_id(data: dict[Any, Any]) -> Result[int, str]:
    """Extract user_id from parsed data."""
    if "user" not in data:
        return Failure("Missing 'user' field")

    if "id" not in data["user"]:
        return Failure("Missing 'id' in user data")

    try:
        user_id = int(data["user"]["id"])
        return Success(user_id)
    except (ValueError, TypeError):
        return Failure("Invalid user ID format")


def validate_user_id(user_id: int) -> Result[int, str]:
    """Validate that user ID is positive."""
    if user_id <= 0:
        return Failure("User ID must be positive")
    return Success(user_id)


def fetch_user_permissions(user_id: int) -> Result[list, str]:
    """Simulate fetching user permissions (could fail due to network, etc.)."""
    # Simulate some users not having permissions
    if user_id == 999:
        return Failure("User not found in permissions database")

    # Simulate network failure for some IDs
    if user_id % 10 == 0:
        return Failure("Network error while fetching permissions")

    # Return mock permissions
    permissions = ["read", "write"] if user_id % 2 == 1 else ["read"]
    return Success(permissions)


def check_write_permission(permissions: list) -> Result[bool, str]:
    """Check if user has write permission."""
    has_write = "write" in permissions
    if not has_write:
        return Failure("User lacks write permission")
    return Success(True)


# =============================================================================
# THE PROBLEM: Pyramid of Doom
# =============================================================================


def process_user_request_nested(json_data: str) -> Result[bool, str]:
    """
    The BAD way: Nested Result handling creating the "Pyramid of Doom"
    """
    parse_result = parse_json(json_data)

    if isinstance(parse_result, Success):
        user_id_result = extract_user_id(parse_result.unwrap())

        if isinstance(user_id_result, Success):
            validation_result = validate_user_id(user_id_result.unwrap())

            if isinstance(validation_result, Success):
                permissions_result = fetch_user_permissions(validation_result.unwrap())

                if isinstance(permissions_result, Success):
                    permission_check = check_write_permission(
                        permissions_result.unwrap()
                    )

                    if isinstance(permission_check, Success):
                        return Success(True)
                    else:
                        return permission_check
                else:
                    return permissions_result
            else:
                return validation_result
        else:
            return user_id_result
    else:
        return parse_result


# =============================================================================
# THE SOLUTION: Using returns library's monadic operations
# =============================================================================


def process_user_request_clean(json_data: str) -> Result[bool, str]:
    """
    The CLEAN way: Using returns library's bind operation for monadic chaining
    """
    return (
        parse_json(json_data)
        .bind(extract_user_id)
        .bind(validate_user_id)
        .bind(fetch_user_permissions)
        .bind(check_write_permission)
    )


def process_user_request_pipeline(json_data: str) -> Result[bool, str]:
    """
    Alternative clean approach: Using flow and pointfree style
    """
    return flow(
        json_data,
        parse_json,
        bind(extract_user_id),
        bind(validate_user_id),
        bind(fetch_user_permissions),
        bind(check_write_permission),
    )

## 1. Composing Monads with `returns.do`

In [ ]:
from returns.result import Result


# Imagine three functions that can fail:
def step1() -> Result[int, str]:
    return Success(10)


def step2(n: int) -> Result[int, str]:
    if n > 5:
        return Success(n * 2)
    return Failure("Step 2 failed: input not > 5")


def step3(n: int) -> Result[str, str]:
    return Success(f"Final result is {n}")


pyramid_result = step1().bind(step2).bind(step3)
pyramid_result

<Success: Final result is 20>

In [78]:
assert isinstance(pyramid_result, Success)
assert pyramid_result.unwrap() == "Final result is 20"
print("✅ `.bind()` chain works as expected.")

✅ `.bind()` chain works as expected.


In [79]:
# --- The clean, flat style with `returns.do` ---
def do_notation_workflow() -> Result[str, str]:
    return Result.do(
        res for first in step1() for sec in step2(first) for res in step3(sec)
    )


do_result = do_notation_workflow()
do_result

<Success: Final result is 20>

In [80]:
assert isinstance(do_result, Success)
assert do_result.unwrap() == "Final result is 20"
assert do_result == pyramid_result  # The results are identical
print("✅ `returns.do` notation produces the same result cleanly.")

✅ `returns.do` notation produces the same result cleanly.


## 2. Collapsing Nested Structures with `sequence`

**The Problem:** You have a list of `Result`s (or `Maybe`s), for example, from validating multiple inputs. You want to know if *all* of them were successful. The ideal outcome is to transform `List[Result[A, B]]` into `Result[List[A], B]`. This new structure would be `Success[List[A]]` if all individual results were `Success`, or the *first* `Failure[B]` encountered.

**The Solution:** `returns.iterables.sequence` does exactly this. It flips the types around.

In [81]:
from returns.iterables import Fold
from returns.result import Result, Success

# --- Verification for Success case ---
all_success = [Success(1), Success(2), Success(3)]
sequenced_success = Fold.collect(all_success, Success(()))

print(f"success sequence: {sequenced_success}")
assert isinstance(sequenced_success, Success)
assert sequenced_success.unwrap() == (1, 2, 3)
print("✅ `sequence` correctly handled a list of all Successes.")

# --- Verification for Failure case ---
one_failure = [
    Success(1),
    Failure("Error at item 2"),
    Success(3),  # This is ignored
    Failure("Error at item 4"),  # This is also ignored
]
sequenced_failure = Fold.collect(one_failure, Success(()))

print(f"failure sequence: {sequenced_failure}")
assert isinstance(sequenced_failure, Failure)
assert sequenced_failure.failure() == "Error at item 2"
print("✅ `sequence` correctly short-circuited on the first Failure.")

success sequence: <Success: (1, 2, 3)>
✅ `sequence` correctly handled a list of all Successes.
failure sequence: <Failure: Error at item 2>
✅ `sequence` correctly short-circuited on the first Failure.


## 3. How to make normal functions work with values in containers

**The Problem:** You have a regular, pure function that works on unwrapped values (e.g., `def add(a: int, b: int) -> int`). You also have your values wrapped in a monadic container like `Maybe` (e.g., `Maybe[int]`). How do you apply your pure function to the wrapped values without manually unwrapping them?

### `apply` values to curried function

In [82]:
from returns.curry import curry
from returns.maybe import Nothing, Some


# A regular, pure function:
@curry
def add(a: int, b: int) -> int:
    return a + b


# Our wrapped values:
maybe_a = Some(10)
maybe_b = Some(5)
maybe_c = Nothing

result_success = maybe_a.apply(maybe_b.apply(Some(add)))
result_success

<Some: 15>

In [83]:
result_success = flow(
    Some(add), lambda _sum: maybe_a.apply(_sum), lambda _sum: maybe_b.apply(_sum)
)
result_success

<Some: 15>

In [97]:
from returns.curry import curry
from returns.io import IO
from returns.pipeline import flow
from returns.pointfree import apply


@curry
def flip(f, a, b):
    return f(b)(a)


rev_apply = flip(apply)


@curry
def sum(a: int, b: int, c: int) -> int:
    return a + b + c


def fetch_balance() -> IO[int]:
    return IO(10)


def fetch_discount() -> IO[int]:
    return IO(-5)


def fetch_price() -> IO[int]:
    return IO(1)


# Pipeline mit flow und pointfree.apply + flip (ohne Lambdas)
result = flow(
    IO(sum),
    rev_apply(fetch_balance()),
    rev_apply(fetch_price()),
    rev_apply(fetch_discount()),
)
result

<IO: 6>

In [ ]:
from returns import pointfree as p

result_success = flow(Some(add), p.apply())
result_success

In [84]:
# --- Verification for Success case ---
assert isinstance(result_success, Some)
assert result_success.unwrap() == 15
print(f"✅ Lifted function correctly produced {result_success}")

✅ Lifted function correctly produced <Some: 15>


In [85]:
# --- Verification for Failure case (if one of the inputs is Nothing) ---
result_failure = maybe_a.apply(maybe_c.apply(Some(add)))
result_failure

<Nothing>

In [99]:
assert result_failure.value_or("nothing") == "nothing"
print(
    f"✅ Lifted function correctly produced {result_failure} when one input was Nothing."
)

✅ Lifted function correctly produced <Nothing> when one input was Nothing.


### `apply` values as a chain

In [90]:
from returns.maybe import Maybe

result_success = Maybe(add).apply(maybe_a).apply(maybe_b)
result_success

AttributeError: 'NoneType' object has no attribute 'apply'

In [ ]:
from returns.curry import curry
from returns.io import IO


@curry
def sum_two_numbers(first: int, second: int) -> int:
    return first + second


one = IO(1)
two = IO(2)
assert IO(sum_two_numbers).apply(one).apply(two) == IO(3)

TypeError: 'int' object is not callable